In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

import numpy as np
b = np.load("b.npy")

height = 64
spatial_norm = 32
b_norm = 100

meta_info = None
log_interval = 100

total_iterations = 50000

decay_iterations = 25000
w_bc = 1000
w_bc_decay = (1 / 1000) ** (1 / decay_iterations)
w_div = 0.1
w_ff = 0.1

FileNotFoundError: [Errno 2] No such file or directory: 'b.npy'

In [ ]:
b_bottom = b[:, :, 0, :]

Nx, Ny, _ = b_bottom.shape
Nz = height

cube_shape = (Nx, Ny, Nz)

def coords(xbounds, ybounds, zbounds):
    return np.stack(np.mgrid[xbounds[0]:xbounds[1]+1, ybounds[0]:ybounds[1]+1, zbounds[0]:zbounds[1]+1], axis=-1)

In [ ]:
bottom_values = b_bottom.reshape(-1, 3)
bottom_values = np.double(bottom_values)
bottom_coords = coords((0, Nx-1), (0, Ny-1), (0, 0)).reshape(-1, 3)
bottom_coords = np.double(bottom_coords)

In [ ]:
# import torch
# from torch import nn
# from torch.utils.data import DataLoader, TensorDataset
# from tqdm import tqdm

# top_lateral_coords = [coords((0, Nx-1), (0, Ny-1), (Nz-2, Nz)),
#                       coords((-1, 1), (0, Ny-1), (0, Nz-1)),
#                       coords((Nx-2, Nx), (0, Ny-1), (0, Nz-1)),
#                       coords((0, Nx-1), (-1, 1), (0, Nz-1)),
#                       coords((0, Nx-1), (Ny-2, Ny), (0, Nz-1))]

# r_top_lateral_shape = [c.shape[:-1] for c in top_lateral_coords]
# r_top_lateral = np.concatenate([c.reshape(((-1, 3))) for c in top_lateral_coords])
# r_bottom = bottom_coords
# bz_bottom = bottom_values[:, 2]

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# class PotentialModel(nn.Module):

#     def __init__(self, b_n, r_p):
#         super().__init__()
#         self.register_buffer('b_n', b_n)
#         self.register_buffer('r_p', r_p)
#         c = np.array([[0, 0, 1/np.sqrt(2*np.pi)]])
#         c = torch.tensor(c, dtype=torch.float64)
#         self.register_buffer('c', c)

#     def forward(self, r):
#         numerator = self.b_n[:, None]
#         denominator = torch.sqrt(torch.sum((r[None, :] - self.r_p[:, None] + self.c[None])**2, -1))
#         potential = torch.sum(numerator/denominator, 0) / (2*np.pi)
#         return potential
    
# pf_batch_size = r_top_lateral.shape[0] // 100
# with torch.no_grad():
#     b_n = torch.tensor(bz_bottom, dtype=torch.float64)
#     r_p = torch.tensor(r_bottom, dtype=torch.float64)
#     model = nn.DataParallel(PotentialModel(b_n, r_p)).to(device)

#     r_coords= torch.tensor(r_top_lateral, dtype=torch.float64)

#     potential = []
#     for r, in tqdm(DataLoader(TensorDataset(r_coords), batch_size=pf_batch_size, num_workers=2),
#                         desc='Potential Boundary'):
#         r = r.to(device)
#         p_batch = model(r)
#         potential += [p_batch.cpu()]

# potential = torch.cat(potential).numpy()

# idx = 0
# fields = []
# for s in r_top_lateral_shape:
#     p = potential[idx:idx + np.prod(s)].reshape(s)
#     b = - 1 * np.stack(np.gradient(p, axis=[0, 1, 2], edge_order=2), axis=-1)
#     fields += [b]
#     idx += np.prod(s)

# pf_fields = [fields[0][:, :, 1].reshape((-1, 3)),
#             fields[1][:, 1, :].reshape((-1, 3)), fields[2][:, 1, :].reshape((-1, 3)),
#             fields[3][1, :, :].reshape((-1, 3)), fields[4][1, :, :].reshape((-1, 3))]
# pf_coords = [top_lateral_coords[0][:, :, 1].reshape((-1, 3)),
#             top_lateral_coords[1][:, 1, :].reshape((-1, 3)), top_lateral_coords[2][:, 1, :].reshape((-1, 3)),
#             top_lateral_coords[3][1, :, :].reshape((-1, 3)), top_lateral_coords[4][1, :, :].reshape((-1, 3))]

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class PotentialModel(nn.Module):

    def __init__(self, b_n, r_p):
        super().__init__()
        self.register_buffer('b_n', b_n)
        self.register_buffer('r_p', r_p)
        c = np.array([[0, 0, 1/np.sqrt(2*np.pi)]])
        c = torch.tensor(c, dtype=torch.float64)
        self.register_buffer('c', c)

    def forward(self, r):
        numerator = self.b_n[:, None]
        denominator = torch.sqrt(torch.sum((r[None, :] - self.r_p[:, None] + self.c[None])**2, -1))
        potential = torch.sum(numerator/denominator, 0) / (2*np.pi)
        return potential


top_lateral_coordinates = [coords((0, Nx-1), (0, Ny-1), (Nz-1, Nz-1)).reshape(-1, 3),
                      coords((0, 0), (0, Ny-1), (0, Nz-1)).reshape(-1, 3),
                      coords((Nx-1, Nx-1), (0, Ny-1), (0, Nz-1)).reshape(-1, 3),
                      coords((0, Nx-1), (0, 0), (0, Nz-1)).reshape(-1, 3),
                      coords((0, Nx-1), (Ny-1, Ny-1), (0, Nz-1)).reshape(-1, 3)]

b_n = torch.tensor(bottom_values[:, 2], dtype=torch.float64)
r_p = torch.tensor(bottom_coords, dtype=torch.float64)

model = nn.DataParallel(PotentialModel(b_n, r_p)).to(device)

pf_fields = []
pf_coords = []
for r_coords in top_lateral_coordinates:
    r_coords = torch.tensor(r_coords, dtype=torch.float64)
    pf_batch_size = int(np.prod(r_coords.shape[:-1]) // 10)

    fields = []
    for r, in tqdm(DataLoader(TensorDataset(r_coords), batch_size=pf_batch_size, num_workers=2),
                        desc='Potential Boundary'):
        r = r.to(device).requires_grad_(True)
        p_batch = model(r)
        b_p = -1 * torch.autograd.grad(p_batch, r, torch.ones_like(p_batch), retain_graph=True, create_graph=True)[0]
        fields += [b_p.clone().detach().cpu().numpy()]
    pf_fields += [np.concatenate(fields)]
    pf_coords += [r_coords.clone().detach().cpu().numpy()]

Potential Boundary: 100%|██████████| 11/11 [00:00<00:00, 94.84it/s]


In [ ]:
# import torch
# from torch import nn
# from torch.utils.data import DataLoader, TensorDataset
# from tqdm import tqdm

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# class PotentialModel(nn.Module):

#     def __init__(self, b_n, r_p):
#         super().__init__()
#         self.register_buffer('b_n', b_n)
#         self.register_buffer('r_p', r_p)
#         c = np.array([[0, 0, 1/np.sqrt(2*np.pi)]])
#         c = torch.tensor(c, dtype=torch.float64)
#         self.register_buffer('c', c)

#     def forward(self, r):
#         numerator = self.b_n[:, None]
#         denominator = torch.sqrt(torch.sum((r[None, :] - self.r_p[:, None] + self.c[None])**2, -1))
#         potential = torch.sum(numerator/denominator, 0) / (2*np.pi)
#         return potential


# top_lateral_coordinates = [coords((0, Nx-1), (0, Ny-1), (Nz-1, Nz-1)),
#                            coords((0, 0), (0, Ny-1), (0, Nz-1)),
#                            coords((Nx-1, Nx-1), (0, Ny-1), (0, Nz-1)),
#                            coords((0, Nx-1), (0, 0), (0, Nz-1)),
#                            coords((0, Nx-1), (Ny-1, Ny-1), (0, Nz-1))]

# top_lateral_coordinate_shape = [np.prod(c.shape[:-1]) for c in top_lateral_coordinates]
# top_lateral_coordinate = np.concatenate([c.reshape(-1, 3) for c in top_lateral_coordinates])

# b_n = torch.tensor(bottom_values[:, 2], dtype=torch.float64)
# r_p = torch.tensor(bottom_coords, dtype=torch.float64)

# model = nn.DataParallel(PotentialModel(b_n, r_p)).to(device)

# pf_fields = []
# pf_coords = []
# for r_coords in top_lateral_coordinate:
#     r_coords = torch.tensor(r_coords, dtype=torch.float64)
#     pf_batch_size = int(np.prod(r_coords.shape[:-1]) // 10)

#     fields = []
#     for r, in tqdm(DataLoader(TensorDataset(r_coords), batch_size=pf_batch_size, num_workers=2),
#                         desc='Potential Boundary'):
#         r = r.to(device).requires_grad_(True)
#         p_batch = model(r)
#         b_p = -1 * torch.autograd.grad(p_batch, r, torch.ones_like(p_batch), retain_graph=True, create_graph=True)[0]
#         fields += [b_p.clone().detach().cpu().numpy()]
#     pf_fields += [np.concatenate(fields)]
#     pf_coords += [r_coords.clone().detach().cpu().numpy()]

In [ ]:
top_lateral_values = np.concatenate(pf_fields) 
top_lateral_coords = np.concatenate(pf_coords)

boundary_values = np.concatenate([top_lateral_values, bottom_values])
boundary_coords = np.concatenate([top_lateral_coords, bottom_coords])

normalized_boundary_values = boundary_values / b_norm
normalized_boundary_coords = boundary_coords / spatial_norm

boundary_data = np.stack([normalized_boundary_coords, normalized_boundary_values], 1)

In [ ]:
class BModel(nn.Module):
    def __init__(self, num_inputs, num_outputs, num_neurons, num_layers):
        super().__init__()
        self.d_in = nn.Linear(num_inputs, num_neurons).double()
        lin = [nn.Linear(num_neurons, num_neurons).double() for _ in range(num_layers)]
        self.linear_layers = nn.ModuleList(lin)
        self.d_out = nn.Linear(num_neurons, num_outputs).double()
        self.activation = torch.sin

    def forward(self, x):
        x = self.activation(self.d_in(x))
        for l in self.linear_layers:
            x = self.activation(l(x))
        B = self.d_out(x)
        return B

In [ ]:
model = BModel(3, 3, 256, 8).to(device)

In [ ]:
from torch.optim.lr_scheduler import ExponentialLR
opt = torch.optim.Adam(model.parameters(), lr=5e-4)
scheduler = ExponentialLR(opt, gamma=(5e-5 / 5e-4) ** (1 / total_iterations))

In [ ]:
collocation_coords = coords((0, Nx-1), (0, Ny-1), (0, Nz-1)).reshape(-1, 3)
normalized_collocation_coords = collocation_coords / spatial_norm
normalized_collocation_coords = torch.tensor(normalized_collocation_coords)

In [ ]:
# shuffled_boundary_data_index = np.random.permutation(boundary_data.shape[0])
# shuffled_boundary_data = boundary_data[shuffled_boundary_data_index]
# batch_size = 1000
# pad = batch_size - shuffled_boundary_data.shape[0] % batch_size
# padded_shuffled_boundary_data = np.concatenate([shuffled_boundary_data, shuffled_boundary_data[:pad]])
# n_batches = padded_shuffled_boundary_data.shape[0] // batch_size
# boundary_batches = np.array(np.split(padded_shuffled_boundary_data, n_batches), dtype=np.double)
# np.save('boundary_batches.npy', boundary_batches)

In [ ]:
base_path = 'results/run'
os.makedirs(base_path, exist_ok=True)

In [ ]:
import logging
from datetime import datetime
log = logging.getLogger()
log.setLevel(logging.INFO)
for hdlr in log.handlers[:]:  # remove all old handlers
    log.removeHandler(hdlr)
log.addHandler(logging.FileHandler("{0}/{1}.log".format(base_path, "info_log")))  # set the new file handler
log.addHandler(logging.StreamHandler())  # set the new console handler

In [ ]:
from torch.utils.data import RandomSampler, Dataset

In [ ]:
class BoundaryDataset(Dataset):
    def __init__(self, batches_path):
        self.batches_path = batches_path

    def __len__(self):
        return np.load(self.batches_path, mmap_mode='r').shape[0]

    def __getitem__(self, idx):
        # lazy load data
        d = np.load(self.batches_path, mmap_mode='r')[idx]
        d = np.copy(d)
        coord, field = d[:, 0],  d[:, 1]
        return coord, field
    
def init_loader(batch_size, boundary_data, num_workers, iterations):
    # shuffle boundary_data
    r = np.random.permutation(boundary_data.shape[0])
    boundary_data = boundary_data[r]
    # adjust to batch size
    pad = batch_size - boundary_data.shape[0] % batch_size
    boundary_data = np.concatenate([boundary_data, boundary_data[:pad]])
    # split boundary_data into batches
    n_batches = boundary_data.shape[0] // batch_size
    batches = np.array(np.split(boundary_data, n_batches), dtype=np.double)
    # store batches to disk
    batches_path = os.path.join(base_path, 'batches.npy')
    np.save(batches_path, batches)
    # create boundary_data loaders
    dataset = BoundaryDataset(batches_path)
    # create loader
    data_loader = DataLoader(dataset, batch_size=None, num_workers=num_workers, pin_memory=True,
                                sampler=RandomSampler(dataset, replacement=True, num_samples=iterations))
    return data_loader, batches_path

class RandomCoordinateSampler():

    def __init__(self, cube_shape, spatial_norm, batch_size, cuda=True):
        self.cube_shape = cube_shape
        self.spatial_norm = spatial_norm
        self.batch_size = batch_size
        self.float_tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

    def load_sample(self):
        random_coords = self.float_tensor(self.batch_size, 3).uniform_()
        random_coords[:, 0] *= self.cube_shape[0] / self.spatial_norm
        random_coords[:, 1] *= self.cube_shape[1] / self.spatial_norm
        random_coords[:, 2] *= self.cube_shape[2] / self.spatial_norm
        return random_coords

In [ ]:
from torch.cuda import get_device_name
n_gpus = torch.cuda.device_count()
device_names = [get_device_name(i) for i in range(n_gpus)]
log.info('Using device: %s (gpus %d) %s' % (str(device), n_gpus, str(device_names)))

Using device: cuda (gpus 1) ['NVIDIA GeForce RTX 3060']


In [ ]:
def jacobian(output, coords):
    jac_matrix = [torch.autograd.grad(output[:, i], coords,
                                      grad_outputs=torch.ones_like(output[:, i]).to(output),
                                      retain_graph=True,
                                      create_graph=True)[0]
                  for i in range(output.shape[1])]
    jac_matrix = torch.stack(jac_matrix, dim=1)
    return jac_matrix

def calculate_loss(b, coords):
    jac_matrix = jacobian(b, coords)
    dBx_dx = jac_matrix[:, 0, 0]
    dBy_dx = jac_matrix[:, 1, 0]
    dBz_dx = jac_matrix[:, 2, 0]
    dBx_dy = jac_matrix[:, 0, 1]
    dBy_dy = jac_matrix[:, 1, 1]
    dBz_dy = jac_matrix[:, 2, 1]
    dBx_dz = jac_matrix[:, 0, 2]
    dBy_dz = jac_matrix[:, 1, 2]
    dBz_dz = jac_matrix[:, 2, 2]
    #
    rot_x = dBz_dy - dBy_dz
    rot_y = dBx_dz - dBz_dx
    rot_z = dBy_dx - dBx_dy
    #
    j = torch.stack([rot_x, rot_y, rot_z], -1)
    jxb = torch.cross(j, b, -1)
    force_loss = torch.sum(jxb ** 2, dim=-1) / (torch.sum(b ** 2, dim=-1) + 1e-7)
    divergence_loss = (dBx_dx + dBy_dy + dBz_dz) ** 2
    return divergence_loss, force_loss

In [ ]:
start_time = datetime.now()
# batch_size = int(np.prod(boundary_data.shape[0]) // 10)
batch_size = 10000
# pad = batch_size - boundary_data.shape[0] % batch_size
# padded_boundary_data = np.concatenate([boundary_data, boundary_data[:pad]])
# padded_boundary_data = torch.tensor(padded_boundary_data)
# dataset = TensorDataset(padded_boundary_data)
# data_loader = DataLoader(dataset, batch_size=batch_size, num_workers=2, shuffle=True)

data_loader, batches_path = init_loader(batch_size, boundary_data, 4, total_iterations)

sampler = RandomCoordinateSampler(cube_shape, spatial_norm, batch_size * 2)

In [ ]:
total_b_diff = []
total_divergence_loss = []
total_force_loss = []
model.train()
for iter, (boundary_coords, b_true) in tqdm(enumerate(data_loader, start=0),
                                                    total=len(data_loader), desc='Training'):
    opt.zero_grad()
    # load input data
    boundary_coords, b_true= boundary_coords.to(device), b_true.to(device)
    random_coords = sampler.load_sample()

    # concatenate boundary and random points
    n_boundary_coords = boundary_coords.shape[0]
    coords = torch.cat([boundary_coords, random_coords], 0)
    coords.requires_grad = True
    
    # forward step
    b = model(coords)

    # if iter == 0:
    #     model.eval()
    #     torch.save({'model': model,
    #         'cube_shape': cube_shape,
    #         'b_norm': b_norm,
    #         'spatial_norm': spatial_norm,
    #         'meta_info': meta_info}, os.path.join(base_path, 'fields_%06d.nf2' % iter))
    #     plot_sample(iter-1, batch_size=batch_size)
    #     model.train()

    # compute boundary loss
    boundary_b = b[:n_boundary_coords]
    b_diff = torch.abs(boundary_b - b_true)
    b_diff = torch.mean(b_diff.pow(2).sum(-1))

    # compute div and ff loss
    divergence_loss, force_loss = calculate_loss(b, coords)

    # reset grad from auto-gradient operation
    opt.zero_grad()
    # compute loss
    (b_diff * w_bc +
        divergence_loss.mean() * w_div +
        force_loss.mean() * w_ff).backward()
    
    if iter == 0:
        model.eval()
        torch.save({'BC_loss': b_diff.detach().cpu().numpy(),
            'lambda_BC': w_bc,
            'divergence_loss': divergence_loss.mean().detach().cpu().numpy(),
            'lambda_div': w_div,
            'force_loss': force_loss.mean().detach().cpu().numpy(),
            'lambda_ff': w_ff,}, os.path.join(base_path, 'loss_%06d.nf2' % iter))
        torch.save({'model': model,
            'cube_shape': cube_shape,
            'b_norm': b_norm,
            'spatial_norm': spatial_norm,
            'meta_info': meta_info}, os.path.join(base_path, 'fields_%06d.nf2' % iter))
        model.train()

    # update step
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
    opt.step()

    # save loss information
    total_b_diff += [b_diff.detach().cpu().numpy()]
    total_divergence_loss += [divergence_loss.mean().detach().cpu().numpy()]
    total_force_loss += [force_loss.mean().detach().cpu().numpy()]

    # update training parameters
    if w_bc > 1:
        w_bc *= w_bc_decay
    if scheduler.get_last_lr()[0] > 5e-5:
        scheduler.step()

Training:   0%|          | 5/50000 [00:17<47:41:37,  3.43s/it]


KeyboardInterrupt: 

In [ ]:
model.train()
for iter, (bc_coords, bc_values) in tqdm(enumerate(data_loader, start=0), total=len(data_loader), desc='Training'):
    # bc_data = bc_data[0]
    # bc_coords = bc_data[:, 0, :].to(device)
    # bc_values = bc_data[:, 1, :].to(device)
    bc_coords = bc_coords.to(device)
    bc_values = bc_values.to(device)

    perm = torch.randperm(normalized_collocation_coords.shape[0])
    idx = perm[:batch_size]
    co_coords = normalized_collocation_coords[idx].to(device)

    r = torch.cat([bc_coords, co_coords], 0)
    r.requires_grad = True 

    B = model(r)
    bc_coord = r[0:bc_coords.shape[0]]
    bc_B = B[0:bc_coords.shape[0]]

    bc_loss = torch.sum((bc_B - bc_values)**2, dim=-1)
    bc_loss = torch.mean(bc_loss)

    co_coord = r[bc_coords.shape[0]:]
    co_B = B[bc_coords.shape[0]:]

    dBx_dr = torch.autograd.grad(co_B[:, 0], co_coord, torch.ones_like(co_B[:, 0]), retain_graph=True, create_graph=True)[0]
    dBy_dr = torch.autograd.grad(co_B[:, 1], co_coord, torch.ones_like(co_B[:, 1]), retain_graph=True, create_graph=True)[0]
    dBz_dr = torch.autograd.grad(co_B[:, 2], co_coord, torch.ones_like(co_B[:, 2]), retain_graph=True, create_graph=True)[0]

    dBx_dx = dBx_dr[:, 0]
    dBx_dy = dBx_dr[:, 1]
    dBx_dz = dBx_dr[:, 2]

    dBy_dx = dBy_dr[:, 0]
    dBy_dy = dBy_dr[:, 1]
    dBy_dz = dBy_dr[:, 2]

    dBz_dx = dBz_dr[:, 0]
    dBz_dy = dBz_dr[:, 1]
    dBz_dz = dBz_dr[:, 2]

    rot_x = dBz_dy - dBy_dz
    rot_y = dBx_dz - dBz_dx
    rot_z = dBy_dx - dBx_dy

    J = torch.stack([rot_x, rot_y, rot_z], -1)
    JxB = torch.cross(J, B, dim=-1)

    divB = dBx_dx + dBy_dy + dBz_dz

    force_free_loss = torch.sum(JxB**2, dim=-1) / (torch.sum(B**2, dim=-1) + 1e-7)
    force_free_loss = torch.mean(force_free_loss)
    divergence_loss = torch.sum((divB)**2, dim=-1)
    divergence_loss = torch.mean(divergence_loss)

    loss = w_bc*bc_loss + w_ff*force_free_loss + w_div*divergence_loss

    if iter == 0:
        model.eval()
        torch.save({'BC_loss': bc_loss.detach().cpu().numpy(),
                    'lambda_BC': w_bc,
                    'divergence_loss': divergence_loss.detach().cpu().numpy(),
                    'lambda_div': w_div,
                    'force_loss': force_free_loss.detach().cpu().numpy(),
                    'lambda_ff': w_ff,
                    'LR':scheduler.get_last_lr()[0]}, 
                    os.path.join(base_path, 'loss_%06d.nf2' % iter))
        torch.save({'model': model,
                    'cube_shape': cube_shape,
                    'b_norm': b_norm,
                    'spatial_norm': spatial_norm,
                    'meta_info': meta_info}, 
                    os.path.join(base_path, 'fields_%06d.nf2' % iter))
        model.train()

    opt.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
    opt.step()

    if (log_interval > 0 and (iter + 1) % log_interval == 0) or (iter == 0):
        # log loss
        log.info('[Iteration %06d/%06d] [loss: %.08f] [bc_loss: %.08f; div_loss: %.08f; ff_loss: %.08f] [w_bc: %f, LR: %f] [%s]' %
                        (iter + 1, total_iterations,
                        loss,
                        w_bc*bc_loss,
                        w_ff*force_free_loss,
                        w_div*divergence_loss,
                        w_bc,
                        scheduler.get_last_lr()[0],
                        datetime.now() - start_time))
        
        model.eval()
        torch.save({'BC_loss': bc_loss.detach().cpu().numpy(),
                    'lambda_BC': w_bc,
                    'divergence_loss': divergence_loss.detach().cpu().numpy(),
                    'lambda_div': w_div,
                    'force_loss': force_free_loss.detach().cpu().numpy(),
                    'lambda_ff': w_ff,
                    'LR':scheduler.get_last_lr()[0]}, 
                    os.path.join(base_path, 'loss_%06d.nf2' % iter))
        torch.save({'model': model,
                    'cube_shape': cube_shape,
                    'b_norm': b_norm,
                    'spatial_norm': spatial_norm,
                    'meta_info': meta_info}, 
                    os.path.join(base_path, 'fields_%06d.nf2' % iter))
        model.train()

    if w_bc > 1:
        w_bc *= w_bc*w_bc_decay
    if scheduler.get_last_lr()[0] > 5e-5:
        scheduler.step()

model.eval()
torch.save({'BC_loss': bc_loss.detach().cpu().numpy(),
            'lambda_BC': w_bc,
            'divergence_loss': divergence_loss.detach().cpu().numpy(),
            'lambda_div': w_div,
            'force_loss': force_free_loss.detach().cpu().numpy(),
            'lambda_ff': w_ff,
            'LR':scheduler.get_last_lr()[0]}, 
            os.path.join(base_path, 'loss_final.nf2'))
torch.save({'model': model,
            'cube_shape': cube_shape,
            'b_norm': b_norm,
            'spatial_norm': spatial_norm,
            'meta_info': meta_info}, 
            os.path.join(base_path, 'fields_final.nf2'))
torch.save({'m': model.state_dict(),
            'o': opt.state_dict(), },
            os.path.join(base_path, 'model_final.pt'))

NameError: name 'model' is not defined